number of cities=20, each city has coordinates x,y generated randomly from 0-100

the distance between 2 cities is Euclidean distance


citi-dict has city name and coordinates

In [2]:
# Imports 
import numpy as np
import random

from datetime import datetime
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Amsterdam': [13, 20],
 'Barcelona': [5, 95],
 'Berlin': [13, 92],
 'Birmingham': [88, 23],
 'Brussels': [71, 74],
 'Bucharest': [63, 27],
 'Budapest': [4, 40],
 'Istanbul': [13, 5],
 'Kyiv': [50, 55],
 'London': [88, 67],
 'Milan': [79, 64],
 'Minsk': [23, 40],
 'Moscow': [68, 49],
 'Munich': [52, 34],
 'Paris': [3, 61],
 'Prague': [43, 50],
 'Rome': [52, 6],
 'Sofia': [4, 30],
 'Vienna': [83, 38],
 'Warsaw': [59, 97]}

**Create the first population set**
We randomly shuffle the cities N times where N=population_size

to create a sngle solution, the choice function is used to create a list of random cities from the cities list

this function is run as many times as the number of population and the sol is appended to the population set

In [3]:
np.random.choice(list(range(n_cities)), n_cities, replace=False)

array([ 1, 14, 10, 17, 15,  8, 11,  0,  6,  7,  5, 19,  4, 12, 13,  2, 16,
        9, 18,  3])

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Paris', 'Berlin', 'Munich', ..., 'Sofia', 'Minsk', 'Prague'],
       ['Kyiv', 'Vienna', 'Birmingham', ..., 'Munich', 'London',
        'Moscow'],
       ['Milan', 'Istanbul', 'Brussels', ..., 'Bucharest', 'London',
        'Barcelona'],
       ...,
       ['Birmingham', 'Istanbul', 'Warsaw', ..., 'Munich', 'Paris',
        'Kyiv'],
       ['Prague', 'Barcelona', 'Amsterdam', ..., 'Birmingham', 'Kyiv',
        'Moscow'],
       ['Barcelona', 'Kyiv', 'Sofia', ..., 'Munich', 'Berlin', 'London']],
      dtype='<U10')

**Evaluate solutions fitness**
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first. The fitness function needs to compute the distance between subsequent cities.

compute the distance between first 2 cities and add to total distance, etc...

fitness_eval function computes fitness for a single sol,,

get_all_fitness function creates a list of all the fitnesses for all the solutions//100 sol

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([1039.24672641,  886.32838256, 1021.26158574, 1018.03706873,
        952.64635639,  966.64377586,  877.90507305,  862.89604339,
       1052.48361698, 1223.28812324, 1038.02345029,  948.89023282,
        918.27278716,  932.69218242,  947.29325303, 1012.38367511,
       1044.29884045,  932.1810088 ,  992.92098785,  982.51508274,
       1062.49775847, 1132.78568926,  965.41230635, 1068.7198901 ,
        963.7822558 ,  974.61489168,  969.6829078 , 1114.34610992,
       1064.50498863,  958.57498662, 1022.91441796,  992.34803426,
       1036.69563832, 1080.99162902, 1088.71689256,  965.43125314,
        990.02383735,  950.74453673, 1086.43685978, 1021.17264747,
       1022.20430123, 1120.06434917,  793.81741272,  934.27804965,
       1074.66536371, 1209.49865745, 1132.95712456, 1056.6283494 ,
       1094.24955249, 1086.0675034 , 1001.9145798 , 1065.73470205,
        947.93070959,  927.79245743,  987.43884997, 1053.22831069,
        919.91194878, 1049.35571355,  935.31245917, 1044.31093

roulette wheel selection,, 

select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [6]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    #the 2 parents selected
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
#the second parent
progenitor_list[0][2]

array(['Munich', 'Barcelona', 'Minsk', 'Rome', 'Amsterdam', 'Sofia',
       'Warsaw', 'Kyiv', 'Berlin', 'Prague', 'Budapest', 'Vienna',
       'Istanbul', 'Birmingham', 'Paris', 'Bucharest', 'London', 'Milan',
       'Moscow', 'Brussels'], dtype='<U10')

**For each pair of parents we'll generate an offspring pair**. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [7]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Kyiv', 'Budapest', 'Paris', 'Vienna', 'Milan', 'London', 'Prague',
       'Istanbul', 'Amsterdam', 'Warsaw', 'Berlin', 'Brussels',
       'Barcelona', 'Bucharest', 'Munich', 'Moscow', 'Rome', 'Sofia',
       'Minsk', 'Birmingham'], dtype='<U10')

Now for each element of the new population we add a random chance of swappingNow for each element of the new population **we add a random chance of swapping**

In [8]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)
#swap 2 cities within the same sol/offspring
        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Istanbul', 'Budapest', 'Prague', 'Vienna', 'Milan', 'London',
       'Sofia', 'Kyiv', 'Amsterdam', 'Warsaw', 'Berlin', 'Brussels',
       'Barcelona', 'Bucharest', 'Birmingham', 'Moscow', 'Rome', 'Paris',
       'Munich', 'Minsk'], dtype='<U10')

In [9]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 793.8174127226965 1005.6408708066483 28/05/22 21:33
100 809.1512445820886 1010.5362308212509 28/05/22 21:33
200 805.9823557309519 1023.1436222255475 28/05/22 21:33
300 674.3442775697696 1015.3202890320015 28/05/22 21:33
400 774.0747185522334 1006.0859456229333 28/05/22 21:33
500 769.343041838473 1019.4980694550079 28/05/22 21:33
600 760.65673971887 1007.901342908117 28/05/22 21:33
700 789.2242418115727 1009.9032257954813 28/05/22 21:33
800 738.1180132025285 1017.3917649383235 28/05/22 21:33
900 763.2109687435003 1017.0608130586155 28/05/22 21:33
1000 817.6412827568513 1009.0893864673588 28/05/22 21:33
1100 791.5148179638782 1014.9646106223054 28/05/22 21:33
1200 821.9952724931258 1021.5746287887799 28/05/22 21:33
1300 746.3276401110301 1014.0228084488305 28/05/22 21:33
1400 803.5453843283816 1007.7215129145155 28/05/22 21:33
1500 813.1580661126956 1016.2259660805128 28/05/22 21:33
1600 780.7756936427676 1009.7104701081729 28/05/22 21:33
1700 778.5018821756381 1022.1741532847205 28/05

best solution is found it iteration 8106, with min fitness of 582.86 and the arrangement of cities is as shown below

In [10]:
best_solution

[8106,
 582.8631947665648,
 array([['Warsaw', 'Milan', 'Brussels', 'Kyiv', 'Prague', 'London',
         'Barcelona', 'Berlin', 'Paris', 'Minsk', 'Amsterdam', 'Moscow',
         'Vienna', 'Birmingham', 'Munich', 'Bucharest', 'Rome',
         'Budapest', 'Sofia', 'Istanbul']], dtype='<U10')]